In [2]:
import numpy as np
from numpy import mean, std
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

from os.path import exists
from os.path import join as fullfile
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.optimizers import SGD, Adam

# Config the memory allocation of the gpu for gpu acceleration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

import pickle

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

1 Physical GPUs, 1 Logical GPUs


In [3]:
model_cnn_sgd = tf.keras.models.load_model('/home/liam/Documents/Projects/cnn/cnnMNIST_trained_sgd/')
model_cnn_adam = tf.keras.models.load_model('/home/liam/Documents/Projects/cnn/cnnMNIST_trained_adam/')
model_nn_sgd = tf.keras.models.load_model('/home/liam/Documents/Projects/nn/nnMNIST_trained_sgd/')
model_nn_adam = tf.keras.models.load_model('/home/liam/Documents/Projects/nn/nnMNIST_trained_adam/')

In [91]:
(_,_),(testX, testy) = tf.keras.datasets.mnist.load_data()
testX = testX.reshape(testX.shape[0], testX.shape[1], testX.shape[2], 1)
enc = OneHotEncoder(sparse=False)
testy_ohe = enc.fit_transform(testy.reshape(-1,1))

cnn_sgd_pred = model_cnn_sgd.predict(testX)

cnn_adam_pred = model_cnn_adam.predict(testX)

nn_sgd_pred = model_nn_sgd.predict(testX)

nn_adam_pred = model_nn_adam.predict(testX)

for i in range(0, len(testy)):
    preds = np.sum((cnn_sgd_pred[i], cnn_adam_pred[i], nn_sgd_pred[i], nn_adam_pred[i]), axis=0)
    ens_vote = np.argmax(preds, axis=0)
    if testy[i] != ens_vote:
        print(f'Example {i} incorrectly classified')
        print(f'Ground truth: {testy[i]} \t Vote: {ens_vote}')
        print('\n')


Example 73 incorrectly classified
Ground truth: 9 	 Vote: 8


Example 247 incorrectly classified
Ground truth: 4 	 Vote: 6


Example 259 incorrectly classified
Ground truth: 6 	 Vote: 0


Example 320 incorrectly classified
Ground truth: 9 	 Vote: 7


Example 321 incorrectly classified
Ground truth: 2 	 Vote: 7


Example 340 incorrectly classified
Ground truth: 5 	 Vote: 3


Example 381 incorrectly classified
Ground truth: 3 	 Vote: 7


Example 445 incorrectly classified
Ground truth: 6 	 Vote: 0


Example 495 incorrectly classified
Ground truth: 8 	 Vote: 2


Example 582 incorrectly classified
Ground truth: 8 	 Vote: 2


Example 659 incorrectly classified
Ground truth: 2 	 Vote: 1


Example 684 incorrectly classified
Ground truth: 7 	 Vote: 3


Example 691 incorrectly classified
Ground truth: 8 	 Vote: 4


Example 813 incorrectly classified
Ground truth: 9 	 Vote: 8


Example 839 incorrectly classified
Ground truth: 8 	 Vote: 3


Example 844 incorrectly classified
Ground truth: 8 	 Vot

In [53]:
ens_pred_stack = np.stack((cnn_sgd_pred, cnn_adam_pred, nn_sgd_pred, nn_adam_pred), axis=2)
ens_pred_sum = np.sum(ens_pred_stack, axis=2) 
ens_pred = np.argmax(ens_pred_sum, axis=1)
np.mean(ens_pred == testy)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 4. 0. 0.]
7


0.977

In [98]:
model_names = ['cnn_sgd', 'cnn_adam', 'nn_sgd', 'nn_adam']
for i in range(0,4):
    fig = plt.figure(figsize=(4,4), dpi = 300)
    pred_temp = np.argmax(ens_pred_stack[:,:,i], axis=1)
    
    conf_mtx = confusion_matrix(y_true=testy, y_pred=pred_temp, normalize='true')
    conf_mtx_disp = ConfusionMatrixDisplay(confusion_matrix=conf_mtx)
    conf_mtx_disp.plot(ax=fig.gca, values_format='.2g')
    plt.title(f'{model_names[i]}, Acc: {np.mean(pred_temp == testy)}')
    plt.show()


AttributeError: 'function' object has no attribute 'figure'

<Figure size 1200x1200 with 0 Axes>

In [99]:
fig.gca

<bound method FigureBase.gca of <Figure size 1200x1200 with 0 Axes>>